In [1]:
import pandas as pd
import sys
import seaborn as sns
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm

In [2]:
df = pd.read_csv(
    "./data/data_reduced.csv",
    usecols={
        "dateid",
        "platform",
        "gamemode",
        "mapname",
        "matchid",
        "roundnumber",
        "objectivelocation",
        "winrole",
        "endroundreason",
        "roundduration",
        "clearancelevel",
        "skillrank",
        "role",
        "team",
        "haswon",
        "operator",
        "nbkills",
        "isdead"
    },
    dtype={
        "dateid": "int64",
        "platform": "category",
        "gamemode": "category",
        "mapname": "category",
        "matchid": "int64",
        "roundnumber": "int64",
        "objectivelocation": "category",
        "winrole": "category",
        "endroundreason": "category",
        "roundduration": "int64",
        "clearancelevel": "int64",
        "skillrank": "category",
        "role": "category",
        "team": "int64",
        "haswon": "boolean",
        "operator": "category",
        "nbkills": "int64",
        "isdead": "boolean"
    },
)

In [3]:
df.shape

(3889573, 18)

In [4]:
df.head()

,dateid,platform,gamemode,mapname,matchid,roundnumber,objectivelocation,winrole,endroundreason,roundduration,clearancelevel,skillrank,role,team,haswon,operator,nbkills,isdead
0,20170212,PC,BOMB,BARTLETT_U.,1529883301,6,ROWING_MUSEUM-TROPHY_ROOM,Attacker,DefendersEliminated,246,147,Platinum,Attacker,1,True,JTF2-BUCK,1,True
1,20170212,PC,BOMB,BARTLETT_U.,1529883301,2,READING_ROOM-LIBRARY,Defender,AttackersEliminated,231,40,Unranked,Defender,0,True,GIGN-ROOK,0,True
2,20170212,PC,BOMB,BARTLETT_U.,1529883301,4,KITCHEN-PIANO_ROOM,Attacker,DefendersEliminated,233,60,Gold,Defender,0,False,G.E.O.-MIRA,0,True
3,20170212,PC,BOMB,BARTLETT_U.,1529883301,6,ROWING_MUSEUM-TROPHY_ROOM,Attacker,DefendersEliminated,246,40,Unranked,Defender,0,False,GIGN-ROOK,0,True
4,20170212,PC,BOMB,BARTLETT_U.,1529883301,7,CLASSROOM-LIBRARY,Attacker,DefendersEliminated,168,56,Unranked,Defender,0,False,SWAT-CASTLE,0,True


In [6]:
def print_size(df):
    print(round(sys.getsizeof(df) / (1024 ** 2), 2), "mb")

print_size(df)

255.96 mb


In [7]:
df["winrole"].value_counts(dropna=False)

Defender    2032656
Attacker    1856917
Name: winrole, dtype: int64

In [8]:
df["mapname"].value_counts()

HOUSE               255100
KAFE_DOSTOYEVSKY    252848
CLUB_HOUSE          250387
CHALET              248992
OREGON              248659
KANAL               247234
HEREFORD_BASE       245882
PLANE               244267
SKYSCRAPER          243444
CONSULATE           240936
BANK                240288
BORDER              238590
BARTLETT_U.         237370
YACHT               237142
FAVELAS             236969
COASTLINE           221465
Name: mapname, dtype: int64

In [10]:
test = pd.read_csv("./data/datadump_s5_summary_objectives.csv")
test.shape

(8785348, 12)

In [21]:
test.head()

,platform,dateid,gamemode,mapname,objectivelocation,skillrank,role,operator,nbwins,nbkills,nbdeaths,nbpicks
0,PC,20170210,BOMB,BANK,EXECUTIVE_LOUNGE-CEO_OFFICE,Bronze,Attacker,G.E.O.-JACKAL,0,0,1,1
1,PC,20170210,BOMB,BANK,EXECUTIVE_LOUNGE-CEO_OFFICE,Bronze,Attacker,GIGN-MONTAGNE,1,0,2,3
2,PC,20170210,BOMB,BANK,EXECUTIVE_LOUNGE-CEO_OFFICE,Bronze,Attacker,GSG9-IQ,0,0,1,1
3,PC,20170210,BOMB,BANK,EXECUTIVE_LOUNGE-CEO_OFFICE,Bronze,Attacker,JTF2-BUCK,0,0,1,1
4,PC,20170210,BOMB,BANK,EXECUTIVE_LOUNGE-CEO_OFFICE,Bronze,Attacker,SAS-THATCHER,0,0,1,1


In [22]:
df_maps = test.groupby(["mapname", "operator"], as_index=False)[['nbwins', 'nbkills', 'nbdeaths', 'nbpicks']].sum()
df_maps

,mapname,role,operator,nbwins,nbkills,nbdeaths,nbpicks
0,BANK,Attacker,BOPE-CAPITAO,43342,59347,65445,90290
1,BANK,Attacker,G.E.O.-JACKAL,117301,179952,168240,239529
2,BANK,Attacker,GIGN-MONTAGNE,50908,43913,77346,109711
3,BANK,Attacker,GIGN-RESERVE,1949,2046,2996,4231
4,BANK,Attacker,GIGN-TWITCH,135596,197271,181915,275564
...,...,...,...,...,...,...,...
635,YACHT,Defender,SPETSNAZ-RESERVE,1886,2230,3362,4257
636,YACHT,Defender,SPETSNAZ-TACHANKA,9582,10868,15477,21121
637,YACHT,Defender,SWAT-CASTLE,52463,59472,77938,107544
638,YACHT,Defender,SWAT-PULSE,89467,125819,134204,180387


In [24]:
df_maps["win_ratio"] = df_maps["nbwins"] / df_maps["nbpicks"]
df_maps

,mapname,role,operator,nbwins,nbkills,nbdeaths,nbpicks,kd_ratio,win_ratio
0,BANK,Attacker,BOPE-CAPITAO,43342,59347,65445,90290,0.480031,0.480031
1,BANK,Attacker,G.E.O.-JACKAL,117301,179952,168240,239529,0.489715,0.489715
2,BANK,Attacker,GIGN-MONTAGNE,50908,43913,77346,109711,0.464019,0.464019
3,BANK,Attacker,GIGN-RESERVE,1949,2046,2996,4231,0.460648,0.460648
4,BANK,Attacker,GIGN-TWITCH,135596,197271,181915,275564,0.492067,0.492067
...,...,...,...,...,...,...,...,...,...
635,YACHT,Defender,SPETSNAZ-RESERVE,1886,2230,3362,4257,0.443035,0.443035
636,YACHT,Defender,SPETSNAZ-TACHANKA,9582,10868,15477,21121,0.453672,0.453672
637,YACHT,Defender,SWAT-CASTLE,52463,59472,77938,107544,0.487828,0.487828
638,YACHT,Defender,SWAT-PULSE,89467,125819,134204,180387,0.495973,0.495973


In [25]:
df_maps["kd_ratio"] = df_maps["nbkills"] / df_maps["nbdeaths"]
df_maps

,mapname,role,operator,nbwins,nbkills,nbdeaths,nbpicks,kd_ratio,win_ratio
0,BANK,Attacker,BOPE-CAPITAO,43342,59347,65445,90290,0.906823,0.480031
1,BANK,Attacker,G.E.O.-JACKAL,117301,179952,168240,239529,1.069615,0.489715
2,BANK,Attacker,GIGN-MONTAGNE,50908,43913,77346,109711,0.567748,0.464019
3,BANK,Attacker,GIGN-RESERVE,1949,2046,2996,4231,0.682911,0.460648
4,BANK,Attacker,GIGN-TWITCH,135596,197271,181915,275564,1.084413,0.492067
...,...,...,...,...,...,...,...,...,...
635,YACHT,Defender,SPETSNAZ-RESERVE,1886,2230,3362,4257,0.663296,0.443035
636,YACHT,Defender,SPETSNAZ-TACHANKA,9582,10868,15477,21121,0.702203,0.453672
637,YACHT,Defender,SWAT-CASTLE,52463,59472,77938,107544,0.763068,0.487828
638,YACHT,Defender,SWAT-PULSE,89467,125819,134204,180387,0.937520,0.495973


In [27]:
df_maps_role = test.groupby(["mapname", "role"], as_index=False)[['nbwins', 'nbkills', 'nbdeaths', 'nbpicks']].sum()
df_maps_role["win_ratio"] = df_maps_role["nbwins"] / df_maps_role["nbpicks"]
df_maps_role["kd_ratio"] = df_maps_role["nbkills"] / df_maps_role["nbdeaths"]
df_maps_role

,mapname,role,nbwins,nbkills,nbdeaths,nbpicks,win_ratio,kd_ratio
0,BANK,Attacker,1291581,1887730,1870105,2675126,0.482811,1.009425
1,BANK,Defender,1447407,1864148,1896745,2699571,0.536162,0.982814
2,BARTLETT_U.,Attacker,1241273,1815176,1852964,2628860,0.472172,0.979607
3,BARTLETT_U.,Defender,1452915,1848806,1829067,2654339,0.547374,1.010792
4,BORDER,Attacker,1341549,1914519,1879312,2667897,0.502849,1.018734
5,BORDER,Defender,1387612,1874116,1929287,2692283,0.515403,0.971403
6,CHALET,Attacker,1477725,1999576,1794663,2694804,0.548361,1.114179
7,CHALET,Defender,1273391,1790878,2010119,2717417,0.468603,0.890931
8,CLUB_HOUSE,Attacker,1402386,1972144,1843063,2696048,0.520164,1.070036
9,CLUB_HOUSE,Defender,1351948,1837548,1979480,2718838,0.497252,0.928298


In [9]:
df.groupby(["matchid", "roundnumber"])["winrole"].first().reset_index(name="winner_team")

KeyboardInterrupt: 

In [ ]:
df.head(1000).groupby(["matchid", "roundnumber"], as_index=False)["winrole"].first()

In [23]:
# df.groupby(["dateid", "matchid", "roundnumber", "team"])["operator"].apply(list).reset_index(name="operators")

KeyboardInterrupt: 